<a href="https://colab.research.google.com/github/sauravakolia/Fastai2/blob/master/02a_sqrt5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
!git clone https://github.com/sauravakolia/Fastai2.git


Cloning into 'Fastai2'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 54 (delta 25), reused 3 (delta 1), pack-reused 0
Unpacking objects: 100% (54/54), done.


In [3]:
import sys
sys.path.append('/content/Fastai2/')

In [4]:
!curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


In [5]:
 !pip install fire

     |████████████████████████████████| 81kB 2.2MB/s 
  Created wheel for fire: filename=fire-0.3.1-py2.py3-none-any.whl size=111005 sha256=780b4de4ba4ba87bbf23575090c27ed1207b4ad1bdaeca3685b6c69f96c4c08e
  Stored in directory: /root/.cache/pip/wheels/c1/61/df/768b03527bf006b546dce284eb4249b185669e65afc5fbb2ac
Successfully built fire


In [6]:
#export
from exp.nb_02 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

ModuleNotFoundError: ignored

In [ ]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [ ]:
x_train,y_train,x_valid,y_valid = get_data()
train_mean,train_std = x_train.mean(),x_train.std()
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [ ]:
x_train = x_train.view(-1,1,28,28)
x_valid = x_valid.view(-1,1,28,28)


In [ ]:
x_train.shape,x_valid.shape

In [ ]:
n,*_=x_train.shape
c=y_train.max()+1
nh=50
n,c

In [ ]:
from torch import nn

In [ ]:
l1 = nn.Conv2d(1, nh, 5)

In [ ]:
x = x_valid[:100]

In [ ]:
l1.weight.shape

In [ ]:
def stats(x):return x.mean(),x.std()

In [ ]:
stats(x)

In [ ]:
stats(l1.weight),stats(l1.bias)

In [ ]:
t=l1(x)

In [ ]:
stats(t)

In [ ]:
nn.init.kaiming_normal_(l1.weight, a=1.)
stats(l1(x))

In [ ]:
import torch.nn.functional as F

In [ ]:
def f1(x,a=0): return F.leaky_relu(l1(x),a)

In [ ]:
nn.init.kaiming_normal_(l1.weight, a=0)
stats(f1(x))

In [ ]:
l1=nn.Conv2d(1,nh,5)
stats(f1(x))

In [ ]:
rec_fs=l1.weight[0,0].numel()

In [ ]:
nf,ni,*_ = l1.weight.shape
nf,ni

In [ ]:
fan_in  = ni*rec_fs
fan_out = nf*rec_fs
fan_in,fan_out

In [ ]:
def gain(a): return math.sqrt(2.0/(1+a**2))

In [ ]:
gain(1),gain(0),gain(0.01),gain(0.1),gain(math.sqrt(5.))

In [ ]:
def kaiming2(x,a, use_fan_out=False):
    nf,ni,*_ = x.shape
    rec_fs = x[0,0].shape.numel()
    fan = nf*rec_fs if use_fan_out else ni*rec_fs
    std = gain(a) / math.sqrt(fan)
    bound = math.sqrt(3.) * std
    x.data.uniform_(-bound,bound)

In [ ]:
kaiming2(l1.weight, a=0);
stats(f1(x))

In [ ]:
kaiming2(l1.weight, a=math.sqrt(5.))
stats(f1(x))

In [ ]:
class Flatten(nn.Module):
  def forward(self,x):return  x.view(-1)

In [ ]:
m=nn.Sequential(nn.Conv2d(1,8, 5,stride=2,padding=2),nn.ReLU(),
                nn.Conv2d(8,16,3,stride=2,padding=1),nn.ReLU(),
                nn.Conv2d(16,32,3,stride=2,padding=1),nn.ReLU(),
                nn.Conv2d(32,1,3,stride=2,padding=1),nn.ReLU(),
                nn.AdaptiveAvgPool2d(1),
                Flatten(),
)

In [ ]:
y = y_valid[:100].float()

In [ ]:
t = m(x)
stats(t)

In [ ]:
nn.init.kaiming_uniform_??

In [ ]:
for l in m:
    if isinstance(l,nn.Conv2d):
        nn.init.kaiming_uniform_(l.weight)
        l.bias.data.zero_()

In [ ]:
t = m(x)
stats(t)

In [ ]:
nn.init.kaiming_normal_??

In [ ]:
%cd "/content/Fastai2/"

In [7]:
!python notebook2script.py 02a_sqrt5.ipynb

python3: can't open file 'notebook2script.py': [Errno 2] No such file or directory
